In [1]:
import os, gc, sys, copy, pickle
from pathlib import Path
import glob
from tqdm.auto import tqdm
tqdm.pandas()

import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from joblib import Parallel, delayed
import multiprocessing as mp

import albumentations as A
import torch
import torch.nn as nn
import torch.optim as optim
import torch.cuda.amp as amp
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torch.utils.data import WeightedRandomSampler
from sklearn.utils.class_weight import compute_class_weight

import timm

import cv2
cv2.setNumThreads(0)
import PIL
import pydicom
import warnings
warnings.filterwarnings("ignore")

In [2]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    torch.manual_seed(SEED)
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(SEED)
        torch.cuda.manual_seed_all(SEED)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
#     os.environ['TF_CUDNN_DETERMINISTIC'] = str(SEED)
#     tf.random.set_seed(SEED)
#     keras.utils.set_random_seed(seed=SEED)
    print('seeding done!!!')

def flush():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()

この設定は、PL-RSNA-2024-Lumbar-Spine-Classificationというプロジェクトで使用するモデルの設定ですね。以下に設定の詳細を説明します：

project_name: プロジェクトの名前です。Lumbar Spineの分類に関連しているようです。

artifact_name: モデルのアーティファクト名です。

load_kernel: カーネルの読み込み設定ですが、現在はNoneです。

load_last: 最後の重みを読み込むかどうかを示すフラグで、Trueに設定されています。

n_folds: クロスバリデーションの分割数です。ここでは5に設定されています。

backbone: 使用するバックボーンモデルの指定です。efficientnet_b0.ra_in1kが使われています。

img_size: 入力画像のサイズです。384x384ピクセルに設定されています。

n_slice_per_c: 1つのコンポーネントあたりのスライス数です。16に設定されています。

in_chans: 入力チャンネル数です。ここでは1に設定されています（グレースケール画像を想定）。

drop_rate: ドロップアウト率です。0に設定されています。

drop_rate_last: 最終層のドロップアウト率です。0.3に設定されています。

drop_path_rate: ドロップパスの割合です。0に設定されています。

p_mixup: Mixupの確率です。0.5に設定されています。

p_rand_order_v1: ランダムオーダーの確率です。0.2に設定されています。

lr: 学習率です。1e-3 (0.001)に設定されています。

out_dim: 出力次元数です。3に設定されています（3クラス分類を想定）。

epochs: 学習エポック数です。15に設定されています。

batch_size: バッチサイズです。8に設定されています。

device: 使用するデバイスです。CUDAが利用可能な場合はGPU、それ以外の場合はCPUを指定しています。

seed: 乱数シードです。2024に設定されています。

これらの設定は、モデルのアーキテクチャや学習方法、データの扱い方などを決定するための重要なパラメータです。







In [3]:
CONFIG = dict(
    project_name = "PL-RSNA-2024-Lumbar-Spine-Classification",
    artifact_name = "rsnaEffNetModel",
    load_kernel = None,
    load_last = True,
    n_folds = 5,
    backbone = "efficientnet_b0.ra_in1k", # tf_efficientnetv2_s_in21ft1k
    img_size = 384,
    n_slice_per_c = 16,
    in_chans = 1,

    drop_rate = 0.,
    drop_rate_last = 0.3,
    drop_path_rate = 0.,
    p_mixup = 0.5,
    p_rand_order_v1 = 0.2,
    lr = 1e-3,

    out_dim = 3,
    epochs = 15,
    batch_size = 4, 
    device = torch.device("cuda") if torch.cuda.is_available() else "cpu",
    seed = 2024
)

seeding(CONFIG['seed'])

seeding done!!!


この設定（CONFIG）は、ある特定のプロジェクト（PL-RSNA-2024-Lumbar-Spine-Classification）でEfficientNetモデルを使用して腰椎の分類を行うためのパラメータを定義しています。

project_name: プロジェクトの名前を示しています。

artifact_name: モデルの保存時に使用するアーティファクト名です。

load_kernel: カーネルの読み込み設定ですが、Noneに設定されています。

load_last: 最後の重みを読み込むかどうかを示すフラグで、Trueに設定されています。

n_folds: クロスバリデーションの分割数です。ここでは5に設定されています。

backbone: 使用するバックボーンモデルの指定です。"efficientnet_b0.ra_in1k"が使用されています。

img_size: 入力画像のサイズです。384x384ピクセルに設定されています。

n_slice_per_c: 1つのコンポーネントあたりのスライス数です。ここでは16に設定されています。

in_chans: 入力チャンネル数です。1に設定されています（グレースケール画像を想定）。

drop_rate: ドロップアウト率です。0に設定されています。

drop_rate_last: 最終層のドロップアウト率です。0.3に設定されています。

drop_path_rate: ドロップパスの割合です。0に設定されています。

p_mixup: Mixupの確率です。0.5に設定されています。

p_rand_order_v1: ランダムオーダーの確率です。0.2に設定されています。

lr: 学習率です。1e-3 (0.001)に設定されています。

out_dim: 出力次元数です。3に設定されています（3クラス分類を想定）。

epochs: 学習エポック数です。15に設定されています。

batch_size: バッチサイズです。8に設定されています。

device: 使用するデバイスです。CUDAが利用可能な場合はGPU、それ以外の場合はCPUが選択されます。

seed: 乱数シードです。2024に設定されています。

これらの設定は、モデルのアーキテクチャ、学習方法、データの処理方法などを制御し、特定のタスクに最適化されたモデルのトレーニングを可能にします。

In [4]:
sample_df = pd.read_csv('DATA_PATH/sample_submission.csv')
test_desc = pd.read_csv('DATA_PATH/test_series_descriptions.csv')
train_desc = pd.read_csv('DATA_PATH/train_series_descriptions.csv')
train_main = pd.read_csv('DATA_PATH/train.csv')

In [5]:
# define the base path for test images
base_path = f'test_imgs'

# function to get image paths for a series
def get_image_paths(row):
    series_path = os.path.join(base_path, str(row['study_id']), str(row['series_id']))
    if os.path.exists(series_path):
        return [
            os.path.join(series_path, f) for f in os.listdir(series_path) if os.path.isfile(os.path.join(series_path, f))
        ]
    return []

# Mapping of series_description to conditions
condition_mapping = {
    'Sagittal T1': {'left': 'left_neural_foraminal_narrowing', 'right': 'right_neural_foraminal_narrowing'},
    'Axial T2': {'left': 'left_subarticular_stenosis', 'right': 'right_subarticular_stenosis'},
    'Sagittal T2/STIR': 'spinal_canal_stenosis'
}

# Create a list to store the expanded rows
expanded_rows = []

# Expand the dataframe by adding new rows for each file path
for index, row in test_desc.iterrows():
    image_paths = get_image_paths(row)
    conditions = condition_mapping.get(row['series_description'], {})
    if isinstance(conditions, str):  # Single condition
        conditions = {'left': conditions, 'right': conditions}
    for side, condition in conditions.items():
        for image_path in image_paths:
            expanded_rows.append({
                'study_id': row['study_id'],
                'series_id': row['series_id'],
                'series_description': row['series_description'],
                'image_path': image_path,
                'condition': condition,
                'row_id': f"{row['study_id']}_{condition}"
            })

# Create a new dataframe from the expanded rows
expanded_test_desc = pd.DataFrame(expanded_rows)

test_data = expanded_test_desc.copy()
test_data['target'] = 0
test_data.head()

,study_id,series_id,series_description,image_path,condition,row_id,target
0,44036939,2828203845,Sagittal T1,test_imgs\44036939\2828203845\1.dcm,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing,0
1,44036939,2828203845,Sagittal T1,test_imgs\44036939\2828203845\10.dcm,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing,0
2,44036939,2828203845,Sagittal T1,test_imgs\44036939\2828203845\11.dcm,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing,0
3,44036939,2828203845,Sagittal T1,test_imgs\44036939\2828203845\12.dcm,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing,0
4,44036939,2828203845,Sagittal T1,test_imgs\44036939\2828203845\13.dcm,left_neural_foraminal_narrowing,44036939_left_neural_foraminal_narrowing,0


In [6]:
label2id = {"Normal/Mild": 0, "Moderate": 1, "Severe": 2}
id2label = {v:k for k,v in label2id.items()}

In [7]:
def load_dicom(path):
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

このload_dicom関数は、DICOM形式の画像ファイルを読み込み、適切な形式で処理しています。以下に関数の詳細を説明します。

load_dicom(path) 関数の詳細
関数の目的:

DICOM形式の画像ファイルを指定されたパスから読み込みます。
読み込んだDICOM画像データを適切な形式に変換して返します。
引数:

path (str): 読み込むDICOMファイルのパス。
処理内容:

pydicom.read_file(path): pydicomライブラリを使用して指定されたパスのDICOMファイルを読み込みます。dicomオブジェクトとして取得します。

dicom.pixel_array: DICOM画像のピクセルデータを取得します。これはNumPy配列として表されます。

data = data - np.min(data): ピクセルデータを最小値で補正し、負の値をゼロにします。これにより、データの範囲が0以上になります。

if np.max(data) != 0:: 最大値が0でない場合、データを最大値で正規化します。これにより、データが0から1の範囲にスケーリングされます。

data = (data * 255).astype(np.uint8): 画像データを8ビット符号なし整数 (uint8) に変換し、0から255の範囲にスケーリングします。これにより、画像データをグレースケールの8ビット符号なし整数形式で表現できます。

return data: 変換されたDICOM画像データを返します。

使用するライブラリ
pydicom: DICOMファイルを読み込むためのライブラリです。DICOMファイルには患者の医療画像や関連情報が含まれています。

numpy (npとしてインポート): 数値計算を行うための基本的なライブラリです。画像データの処理や変換に使用されます。

In [8]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None, label_name='target'):
        self.dataframe = dataframe
        self.transform = transform
        self.label = dataframe.loc[:, label_name]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        image_path = self.dataframe['image_path'][index]
        image = load_dicom(image_path)  # Define this function to load your DICOM images
        target = self.dataframe['target'][index]
        
        if self.transform:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
            image = self.transform(image=image)['image']
            image = image.transpose(2, 0, 1).astype(np.float32) / 255.

        return image, torch.tensor(target).float()
    
    def get_labels(self):
        return self.label

__init__ メソッド:

dataframe: 入力データを保持するPandas DataFrame。
transform: 前処理を実行するためのTransform関数（例えば、画像のリサイズや正規化など）。
label_name: ラベル列の名前。デフォルトは 'target' です。
self.dataframe: 入力データを保持するDataFrameをクラスのインスタンス変数として格納します。
self.transform: 前処理関数が与えられていれば、そのままインスタンス変数として格納します。
self.label: 指定されたラベル列を取得し、self.label として保持します。

__len__ メソッド:

データセットのサイズ（データ数）を返します。

__getitem__ メソッド:

指定されたインデックス index に対応するデータを返します。
DICOM画像のパスを取得し、load_dicom 関数を使ってDICOM画像を読み込みます。
データフレームからターゲットラベルを取得します。
self.transform が指定されている場合は、画像に対して指定された前処理を適用します（ここではOpenCVを使用してグレースケールからBGRに変換し、指定されたTransform関数を適用しています）。
画像データをPyTorchのテンソルに変換して返し、対応するターゲットラベルも返します。

get_labels メソッド:

データセット全体のラベルを返します。

使用するライブラリ
torch: PyTorchライブラリ。
torchvision: PyTorchの画像処理用ライブラリ。
cv2 (OpenCV): 画像の読み込み、変換に使用されます。
numpy (npとしてインポート): 数値計算を行うための基本的なライブラリ。

In [9]:
def get_transforms(height, width):
    train_tsfm = A.Compose([
        # Geometric augmentations
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=(-30, 30), p=0.5),  # 回転の角度の範囲をタプルで指定する
        
        A.Resize(height=height, width=width),
    ])
    
    valid_tsfm = A.Compose([
        A.Resize(height=height, width=width),
    ])
    
    return {"train": train_tsfm, "eval": valid_tsfm}



def get_dataloaders(data, cfg, split="train"):
    img_size = cfg['img_size']
    height, width = img_size, img_size
    tsfm = get_transforms(height=height, width=width)
    if split == 'train':
        tr_tsfm = tsfm['train']
        ds = CustomDataset(data, transform=tr_tsfm)
        labels = ds.get_labels()
#         class_weights = torch.tensor(compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels))
        class_weights = torch.tensor([1, 2, 4])
        samples_weights = class_weights[labels]
#         print(class_weights)
        sampler = WeightedRandomSampler(weights=samples_weights, 
                                        num_samples=len(samples_weights), 
                                        replacement=True)

        dls = DataLoader(ds, 
                         batch_size=cfg['batch_size'], 
                         sampler=sampler, 
                         num_workers=os.cpu_count(), 
                         drop_last=True, 
                         pin_memory=True)
        
    elif split == 'valid' or split == 'test':
        eval_tsfm = tsfm['eval']
        ds = CustomDataset(data, transform=eval_tsfm)
        dls = DataLoader(ds, 
                         batch_size=2*cfg['batch_size'], 
                         shuffle=False, 
                         num_workers=os.cpu_count(), 
                         drop_last=False, 
                         pin_memory=True)
    else:
        raise Exception("Split should be 'train' or 'valid' or 'test'!!!")
    return dls

このコードは、データセットのトランスフォームやデータローダーを作成する関数を定義しています。以下に関数の詳細を説明します。

get_transforms(height, width): 画像のトランスフォーメーションを定義する関数です。

train_tsfm: 学習時のデータ拡張（Augmentation）を定義します。水平方向と垂直方向のフリップ、ランダムな角度での回転、指定されたサイズへのリサイズを行います。

valid_tsfm: 検証時およびテスト時のデータセットのリサイズを行うトランスフォーメーションです。

A.Compose([...]): albumentationsライブラリを使用して複数の画像変換を組み合わせています。これにより、データセットの多様性を増やし、モデルの汎化性能を向上させます。


get_dataloaders(data, cfg, split="train"): データセットをロードするための関数です。指定されたデータを元に、学習用、検証用、またはテスト用のデータローダーを作成します。

split: 'train', 'valid', 'test' のいずれかで、データセットの使用目的を指定します。

tr_tsfmやeval_tsfm: get_transforms 関数で定義したトランスフォーメーションを取得します。

CustomDataset(data, transform=tr_tsfm): CustomDataset クラスを使用して、指定されたデータとトランスフォーメーションを適用したデータセットを作成します。

WeightedRandomSampler: 各クラスのサンプル数に基づいて重み付けされたランダムサンプリングを行うためのサンプラーです。クラスの不均衡を考慮して学習を効果的に行うために使用されます。

DataLoader: PyTorchのデータローダーを作成し、ミニバッチでデータを読み込みます。ここでは、並列処理を行うためにnum_workersをCPUのコア数として指定し、pin_memory=Trueでメモリピンを有効にしています。


モデル

In [10]:
class TimmModel(nn.Module):
    def __init__(self, backbone, pretrained=False):
        super(TimmModel, self).__init__()

        self.encoder = timm.create_model(
            backbone,
            num_classes=CONFIG["out_dim"],
            features_only=False,
            drop_rate=CONFIG["drop_rate"],
            drop_path_rate=CONFIG["drop_path_rate"],
            pretrained=pretrained
        )

        if 'efficient' in backbone:
            hdim = self.encoder.conv_head.out_channels
            self.encoder.classifier = nn.Identity()
        elif 'convnext' in backbone:
            hdim = self.encoder.head.fc.in_features
            self.encoder.head.fc = nn.Identity()


        self.lstm = nn.LSTM(hdim, 256, num_layers=2, dropout=CONFIG["drop_rate"], bidirectional=True, batch_first=True)
        self.head = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.Dropout(CONFIG["drop_rate_last"]),
            nn.LeakyReLU(0.1),
            nn.Linear(256, CONFIG["out_dim"]),
        )

    def forward(self, x):
        feat = self.encoder(x)
        feat, _ = self.lstm(feat)
        feat = self.head(feat)
        return feat

In [11]:
FLIPS = [None, [-1], [-2], [-2, -1]]

def inference_loop(model, loader):
    model.to(CONFIG["device"])
    model.eval()
    preds = np.empty((0, 3))
    with torch.no_grad():
        for batch in tqdm(loader):
            images, labels = batch
            images = images.to(CONFIG["device"], non_blocking=True)
            with torch.autocast(device_type="cuda", dtype=torch.float16):
#                 logits = model(images.to(torch.float32))
                logits = model(images)
#                 logits = logits.mean(axis=1).softmax(dim=-1)
                logits = logits.softmax(dim=-1)
                preds = np.concatenate([preds, logits.detach().cpu().numpy()])
    np.save('preds.npy', preds)
    
    
def tta_inference_loop(model, loader):
    model.to(CONFIG["device"])
    model.eval()
    preds = np.empty((0, 3))
    with torch.no_grad():
        for batch in tqdm(loader):
            images, labels = batch
            images = images.to(CONFIG["device"], non_blocking=True)
            pred_tta = []
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                for f in FLIPS:
                    logits = model(torch.flip(images, f) if f is not None else images)
                    logits = logits.softmax(dim=-1)
                    pred_tta.append(logits.detach().cpu().numpy())
#                 preds = np.concatenate([preds, logits.detach().cpu().numpy()])
                preds = np.concatenate([preds, np.mean(pred_tta, 0)])
    np.save('preds.npy', preds)
#     return preds

inference_loop(model, loader) 関数

model.to(CONFIG["device"]): モデルをGPU（cuda）またはCPUに移動します。CONFIG["device"]は事前に定義されたデバイスです。

model.eval(): モデルを評価モードに設定します。これにより、ドロップアウトやバッチ正規化などの層の挙動がテストモードとなります。

torch.no_grad(): 勾配計算を無効にします。推論時には勾配は不要なため、メモリを節約します。

for batch in tqdm(loader):: データローダーからバッチごとにデータを取得します。tqdmは進捗バーを表示するためのライブラリです。

images, labels = batch: バッチから画像データと対応するラベルを取得します。

images.to(CONFIG["device"], non_blocking=True): 画像データをGPUに移動します。non_blocking=Trueは非同期処理を意味し、メモリの効率を向上させます。

with torch.autocast(device_type="cuda", dtype=torch.float16):: 半精度での推論を行います。これにより、計算速度が向上し、GPUメモリの使用量が削減されます。

logits = model(images): モデルに画像データを入力し、ロジット（未活性化の出力）を取得します。

logits.softmax(dim=-1): ロジットをソフトマックス関数で確率に変換します。dim=-1は最後の次元でソフトマックスを計算することを意味します。

preds = np.concatenate([preds, logits.detach().cpu().numpy()]): ロジットをnumpy配列に変換し、predsに追加します。detach()はTensorから計算グラフを切り離し、cpu().numpy()でCPU上のnumpy配列に変換します。

np.save('preds.npy', preds): 予測結果をpreds.npyというファイルに保存します。

tta_inference_loop(model, loader) 関数

for f in FLIPS:: TTA（Test Time Augmentation）の各操作についてループします。FLIPSは水平、垂直方向へのフリップの組み合わせを示すリストです。

torch.flip(images, f): 画像データにTTAを適用します。fがNoneでない場合は、指定された軸で画像をフリップします。

pred_tta.append(logits.detach().cpu().numpy()): TTAで得られた予測結果をリスト pred_tta に追加します。

preds = np.concatenate([preds, np.mean(pred_tta, 0)]): TTAで得られた各予測結果の平均を取り、predsに追加します。これにより、複数のデータ拡張による予測を平均してアンサンブル効果を得ます。

これらの関数を使えば、モデルの推論を効率的に行い、さまざまなデータ拡張技術を組み合わせて精度向上を図ることができます。

In [12]:
weights_path = "model_weights.pth"
weights = torch.load(weights_path, map_location=torch.device("cpu"))
model = TimmModel(backbone=CONFIG["backbone"], pretrained=False)
model.load_state_dict(weights)

<All keys matched successfully>

In [13]:
from torch.utils.data import DataLoader

# DataLoaderを直接初期化し、num_workersを0に設定する
dataloader = DataLoader(dataset=test_data, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0)


In [14]:
dls = get_dataloaders(test_data, CONFIG, split="test")
# inference_loop(model, dls)
tta_inference_loop(model, dls)
# _ = Parallel(n_jobs=mp.cpu_count())(
#     delayed(inference_loop(model, dls))
# )

preds = np.load('preds.npy')

  0%|          | 0/25 [00:01<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 3680, 4652, 7900, 12644, 9884, 1536, 1776, 8916, 8952, 9396, 5448, 9280) exited unexpectedly